In [1]:
import numpy as np
import requests
import httpx 
from PIL import Image
import io
from tqdm.auto import tqdm
from tqdm.asyncio import tqdm_asyncio
import onnxruntime as ort

In [2]:
!uv pip install aiohttp

Using Python 3.11.11 environment at: /home/chris/repos/deepweeds-kerascv/.venv
Audited 1 package in 47ms


In [3]:
BATCH_SIZE=4096

In [4]:
def gen_batch(n_batch: int):
    images = []
    for _ in range(n_batch):
        img_bytes = io.BytesIO()
        data = np.random.uniform(0, 255, (256,256,3)).astype(np.uint8)
        Image.fromarray(data).save(img_bytes, "JPEG")
        img_bytes.seek(0)
        images.append(img_bytes)
    return images

## 1. Get baselines running inference directly

In [5]:
onnx_session = ort.InferenceSession('../models/MeNet.onnx', providers=["TensorrtExecutionProvider"])
onnx_input_name = onnx_session.get_inputs()[0].name
onnx_output_name = onnx_session.get_outputs()[0].name
preload_sample = np.random.uniform(0, 255, (1, 256, 256, 3)).astype(np.float32)
onnx_session.run([onnx_output_name], {onnx_input_name: preload_sample})

[array([[1.5377483e-22, 1.1283496e-08, 8.8898247e-05, 1.1581302e-30,
         4.4756601e-18, 2.5863977e-30, 1.1210388e-43, 5.5253008e-07,
         9.9991047e-01]], dtype=float32)]

In [6]:
%%timeit
preload_sample = np.random.uniform(0, 255, (1, 256, 256, 3)).astype(np.float32)
_ = onnx_session.run([onnx_output_name], {onnx_input_name: preload_sample})

1.9 ms ± 54 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
images = [np.random.uniform(0, 255, (1, 256, 256, 3)).astype(np.float32) for _ in range(BATCH_SIZE)] 
for img in tqdm(images):
    _ = onnx_session.run([onnx_output_name], {onnx_input_name: img})

  0%|          | 0/4096 [00:00<?, ?it/s]

## 2. Results running inference through our fastapi endpoint

In [4]:
%%timeit

img_bytes = io.BytesIO()
data = np.random.uniform(0, 255, (256,256,3)).astype(np.uint8)
Image.fromarray(data).save(img_bytes, "JPEG")
img_bytes.seek(0)

url = "http://localhost:8000/predict"
files = {"file": ("image.jpg", img_bytes, "image/jpeg")}

response = requests.post(url, files=files)
assert response.status_code == 200
# print(response.status_code)
# print(response.json())


5.71 ms ± 106 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
async def send_image_request(image_bytes, client: httpx.AsyncClient):
    # Convert numpy array to image bytes
    files = {'file': ('image.jpg', image_bytes, 'image/jpeg')}
    response = await client.post("/predict", files=files)
    return response.json()

In [ ]:
images = gen_batch(BATCH_SIZE // 4)
async with httpx.AsyncClient(base_url="http://localhost:8000/", limits=httpx.Limits(max_connections=1024)) as client:
    res = await tqdm_asyncio.gather(*[send_image_request(image, client) for image in images[:]])

  0%|          | 0/1024 [00:00<?, ?it/s]

100%|██████████| 1024/1024 [00:09<00:00, 109.24it/s]


HTTPX is borked, I've no idea why it's so slow?? - doesn't make any requests with > 1024 coroutines even with a reasonable connection limit (so need to do some extra concurrency control probably), and is worse on both sync and async compared to standard requests.

In [8]:
images = gen_batch(BATCH_SIZE)

In [9]:
for img_bytes in tqdm(images):
    files = {"file": ("image.jpg", img_bytes, "image/jpeg")}
    response = requests.post("http://localhost:8000/predict", files=files)
    assert response.status_code == 200, (response.status_code, response.content)

  0%|          | 0/4096 [00:00<?, ?it/s]

In [ ]:
import aiohttp
async def aio_request(img_bytes, session): 
    data = aiohttp.FormData()
    data.add_field('file', 
                    img_bytes, 
                    filename='image.jpg',
                    content_type='image/jpeg')
    
    async with session.post("http://localhost:8000/predict", data=data) as response:
        return await response.json()

In [ ]:

images = gen_batch(BATCH_SIZE)
async with aiohttp.ClientSession() as session:
    res = await tqdm_asyncio.gather(*[aio_request(image, session) for image in images[:]])

100%|██████████| 4096/4096 [00:11<00:00, 344.50it/s]


Looks like our gpu usage is never above ~50% even through the high request rate, so data throughput / python overhead is a hard-limit - we can't feed the model fast enough.  Bumping the number of processes will ease this but annoyingly we need a separate model instance for each, maybe this would be not be a problem if we switched to a language that allows parallel threads

Running the server with 4x uvicorn workers:

In [13]:
images = gen_batch(BATCH_SIZE)
async with aiohttp.ClientSession() as session:
    res = await tqdm_asyncio.gather(*[aio_request(image, session) for image in images[:]])

100%|██████████| 4096/4096 [00:04<00:00, 833.67it/s]


^ Halves the runtime but 4x's the vram usage (way under the limit anyway though so we're fine, but for sure not ideal with larger models)

Ideas to experiment with in random order:

1. Try an optimised inference service that does clever things I don't have time to implement - (nvidia Triton, TFServing...)
2. Try to figure out how to make the onnx inference session async compatible
3. If no luck try to re-write inference service in cpp with threading & async (as opposed to async only) on the prediction endpoint